In [1]:
from tinyllava.model import *

/home/gjiajun/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/gjiajun/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/gjiajun/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2025-06-30 14:36:01,353] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [28]:
class VQ_Encoder(nn.Module):
    def __init__(self,config,codebook):
        super(VQ_Encoder,self).__init__()
        self.embed_dim=config.embed_dim
        self.discrete_size=config.discrete_size
        self.linear=nn.Sequential(nn.Linear(config.embed_dim,config.discrete_size),nn.GELU())
        self.codebook=codebook
        self.mse=nn.MSELoss()
    def forward(self,inputs_embeds):
        y=self.linear(inputs_embeds)
        y=F.softmax(y,dim=-1)
        y_hard=torch.argmax(y,dim=-1)
        y_hard=torch.nn.functional.one_hot(y_hard, num_classes=self.discrete_size)
        y_diff=y_hard-y
        c=torch.matmul(y_hard.float(),self.codebook)
        L_comm=self.mse(inputs_embeds,c.detach())
        L_code=self.mse(c,inputs_embeds.detach())
        return y_diff.detach()+y,L_comm,L_code

class VQ_Decoder(nn.Module):    
    def __init__(self,config,codebook):
        super(VQ_Decoder,self).__init__()
        self.codebook=codebook
        self.linear=nn.Sequential(nn.Linear(config.embed_dim,config.embed_dim),nn.GELU())
    def forward(self,y):
        outputs_embeds=torch.matmul(y,self.codebook)
        return(self.linear(outputs_embeds))
    
class VQ_VAE(nn.Module):
    def __init__(self,config):
        super(VQ_VAE,self).__init__()
        self.codebook=nn.Parameter(torch.rand([discrete_size,embed_dim]))
        self.encoder=VQ_Encoder(config,self.codebook)
        self.decoder=VQ_Decoder(config,self.codebook)
    def forward(self,inputs_embeds):
        y,L_comm,L_code=self.encoder(inputs_embeds)
        output_embeds=self.decoder(y)
        return(output_embeds,L_comm,L_code)
    
class VQ_config:
    def __init__(self,embed_dim,discrete_size,nheads=10,dropout=0.1,num_layers=8):
        self.embed_dim=embed_dim
        self.discrete_size=discrete_size
        self.nheads=nheads
        self.dropout=dropout
        self.num_layers=num_layers

In [29]:
L_recon_F=nn.MSELoss()
config=VQ_config(100,20)
VQ=VQ_VAE(config)
inputs=torch.rand([4,50,100])
outputs,L1,L2=VQ(inputs)
L_recon=L_recon_F(inputs,outputs)

torch.Size([4, 50, 100])


tensor(0.3400, grad_fn=<MseLossBackward0>)

tensor(0.1753)